In [1]:
import requests as rq
import json
import pandas as pd
pd.set_option('display.precision', 4,
              'display.colheader_justify', 'center')
import numpy as np
import warnings
import pytz
import datetime
import time
from IPython.display import clear_output

In [2]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [3]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [4]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

In [5]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

## Get list of networks

In [6]:
networks_list_response = get_response("/onchain/networks",
                                      use_pro, 
                                      "",
                                      PRO_URL)

In [7]:
networks_list_response["data"][0]

{'id': 'eth',
 'type': 'network',
 'attributes': {'name': 'Ethereum', 'coingecko_asset_platform_id': 'ethereum'}}

In [8]:
df_net = pd.DataFrame(networks_list_response["data"])

## Get DEX for a specific network

In [9]:
network = "eth"

dex_list_response = get_response(f"/onchain/networks/{network}/dexes",
                                 use_pro, 
                                 "",
                                 PRO_URL)

In [10]:
df_dex = pd.DataFrame(dex_list_response["data"])

## Create URLs

In [11]:
def get_url(url_type, network, dex = "", pool_address = ""):   

    url_dict = {
        "trending_pools": f"/onchain/networks/{network}/trending_pools",
        "top_pools": f"/onchain/networks/{network}/pools",
        "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools",
        "specific_pool_dex": f"/onchain/networks/{network}/pools/{pool_address}"
    }

    return url_dict[url_type]

## Get trending pools for a specific network

In [12]:
def collect_response(list_response):    

    response_all = []

    for response in list_response["data"]:
        all_attributes = response["attributes"]
        daily_tx = all_attributes["transactions"]["h24"]
        rel = response["relationships"]
        
        temp_dict = dict(
            pair = all_attributes["name"],
            dex = rel["dex"]["data"]["id"],
            add = all_attributes["address"],
            fdv_usd = all_attributes["fdv_usd"],
            market_cap_usd = all_attributes["market_cap_usd"],
            daily_volume = all_attributes["volume_usd"]["h24"],
            daily_price_change = all_attributes["price_change_percentage"]["h24"],
            daily_buys = daily_tx["buys"],
            daily_sells = daily_tx["sells"],
            daily_buyers = daily_tx["buyers"],
            daily_sellers = daily_tx["sellers"]
        )
        
        response_all.append(temp_dict)

    return response_all

In [13]:
def get_trending_pools(network, sort_by_col):

    target_url = get_url("trending_pools", network)

    trendpool_list_response = get_response(target_url,
                                           use_pro, 
                                           "",
                                           PRO_URL)
    
    trendpool_all = collect_response(trendpool_list_response)    

    return pd.DataFrame(trendpool_all).sort_values(by = [f"{sort_by_col}"],
                                                   ascending = False)

In [14]:
#get_trending_pools("eth", "daily_volume")

## Get top pools for a specific network

In [15]:
def get_top_pools_network(network, sort_by_col):

    target_url = get_url("top_pools", network)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [16]:
#get_top_pools_network("eth", "daily_volume")

## Get top pools for a specific DEX

In [17]:
def get_top_pools_dex(network, dex, sort_by_col):

    target_url = get_url("top_pools_dex", network, dex)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [57]:
get_top_pools_dex("eth", "uniswap_v3", "market_cap_usd")

,pair,dex,add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
4,APE / WETH 0.3%,uniswap_v3,0xac4b3dacb91461209ae9d41ec517c2b9cb1b7daf,1179967758,829486735.092453,4878708.06877839,50.5,2053,1438,1431,460
2,WETH / USDT 0.01%,uniswap_v3,0xc7bbec68d12a0d1830360f8ec58fa599ba1b0e9b,7820949618,7826958545.82488,70566320.279927,-0.21,2691,2332,1343,1609
6,WETH / USDC 0.05%,uniswap_v3,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,7820949618,7826958545.82488,33165588.561879,-0.21,1017,1487,623,1094
11,WETH / USDT 0.05%,uniswap_v3,0x11b815efb8f581194ae79006d24e0d814b7697f6,7820949618,7826958545.82488,7462663.25091344,-0.21,690,507,445,332
17,EIGEN / WETH 0.3%,uniswap_v3,0xc2c390c6cd3c4e6c2b70727d35a45e8a072f18ca,6157926702,681297768.047801,5881497.35814867,2.02,188,542,69,403
8,SPX / WETH 0.3%,uniswap_v3,0x7c706586679af2ba6d1a9fc2da9c6af59883fdd3,570656324,573107857.39551,5378609.45755783,-4.64,861,546,551,277
13,VOLT / TITANX 1%,uniswap_v3,0x3f1a36b6c946e406f4295a89ff06a5c7d62f2fe2,54873680,54131297.1177939,2967297.41280899,34.34,658,250,257,165
12,PUFFER / WETH 1%,uniswap_v3,0xc5c9a9ab6403cdba9722463000146c18b504f0ba,398398417,40778167.8227924,178960.620814305,6.9,76,996,17,965
19,KLAUS / WETH 1%,uniswap_v3,0x7d7b27d4eb74f7728c53a5a4f9d4e82a3652bbe7,38326928,37734367.0142879,977596.466255237,3.7,429,223,343,166
5,USDC / WETH 0.01%,uniswap_v3,0xe0554a476a092703abdb3ef35c80e0d76d32939f,25817765555,35029944897.6172,43948314.6036065,-0.16,1539,1342,1085,712


## Get data for a specific pool address

In [19]:
def collect_pool_response(list_response):    

    response = list_response["data"]
    all_attributes = response["attributes"]
    daily_tx = all_attributes["transactions"]["h24"]
    rel = response["relationships"]
        
    response_dict = dict(
        pair = all_attributes["name"],
        dex = rel["dex"]["data"]["id"],
        add = all_attributes["address"],
        fdv_usd = all_attributes["fdv_usd"],
        market_cap_usd = all_attributes["market_cap_usd"],
        daily_volume = all_attributes["volume_usd"]["h24"],
        daily_price_change = all_attributes["price_change_percentage"]["h24"],
        daily_buys = daily_tx["buys"],
        daily_sells = daily_tx["sells"],
        daily_buyers = daily_tx["buyers"],
        daily_sellers = daily_tx["sellers"]
    )

    return response_dict

In [20]:
def get_pool_data(network, dex, pool_address):

    target_url = get_url("specific_pool_dex", network, dex, pool_address)

    pool_list_response = get_response(target_url,
                                      use_pro, 
                                      "",
                                      PRO_URL)

    pool_all = collect_pool_response(pool_list_response)   

    return pool_all

In [58]:
get_pool_data("eth", "uniswap_v3", "0xc7bbec68d12a0d1830360f8ec58fa599ba1b0e9b")

{'pair': 'WETH / USDT 0.01%',
 'dex': 'uniswap_v3',
 'add': '0xc7bbec68d12a0d1830360f8ec58fa599ba1b0e9b',
 'fdv_usd': '7820954031',
 'market_cap_usd': '7828631363.3987',
 'daily_volume': '70780141.7155127',
 'daily_price_change': '-0.21',
 'daily_buys': 2691,
 'daily_sells': 2334,
 'daily_buyers': 1347,
 'daily_sellers': 1612}

## Uniswap class

In [31]:
from uniswap import Uniswap

In [53]:
# Get wallet private key
def get_private_key():
    f = open("/home/vikas/Documents/MetaMask_private_key.json")
    key_dict = json.load(f)
    return key_dict["private_key"]

In [54]:
# Use None for address and private_key when not doing a transaction

address = "0xAf418C54351BA8a0Aa15Ba4A5C99C46C122B3DBC"      
private_key = get_private_key()    
version = 3
provider = "https://rpc.ankr.com/eth"

uniswap = Uniswap(address = address,
                  private_key = private_key,
                  version = version,
                  provider = provider)

In [64]:
# Token contract address
# https://support.uniswap.org/hc/en-us/articles/26757826138637-What-is-a-token-contract-address

eth = "0x0000000000000000000000000000000000000000"
bat = "0x0D8775F648430679A709E98d2b0Cb6250d2887EF"
dai = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
usdt = "0xdAC17F958D2ee523a2206206994597C13D831ec7"

In [68]:
# Another wallet address
mm_add_2 = "0x758A71BDbb79a9FFca78C2eB82459985f6cffAFC"

#### Get swap prices

In [71]:
# Returns the amount of ETH you need to pay (in wei) to get X amount of USDT

uniswap.get_price_output(eth, usdt, 3 * 10**6, fee = 100) / (10**18)

0.001135217163866888

#### Check balance

In [74]:
uniswap.get_eth_balance() / 10**18

0.004425315035114578

In [76]:
uniswap.get_token_balance(usdt) / 10**6

3.0

#### Make trade

In [73]:
#uniswap.make_trade_output(eth, usdt, 3 * 10**6, fee = 100)